In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

%load_ext lab_black

In [ ]:
import os.path

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import h5py
import numpy as np

from astropy.time import Time
from astropy.time import TimeDelta
from astropy import units as u
import matplotlib.dates as mdates
from matplotlib.ticker import FormatStrFormatter
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
from matplotlib import ticker
from scipy.signal import stft

from lsst_efd_client import EfdClient
from lsst.summit.utils.tmaUtils import TMAEventMaker, TMAState, TMAEvent
from lsst.summit.utils.efdUtils import getEfdData, makeEfdClient

In [ ]:
# Utility functions
key_m1m3_dict = {
    "1 X": "m1m3_x_1",
    "1 Y": "m1m3_y_1",
    "1 Z": "m1m3_z_1",
    "2 X": "m1m3_x_2",
    "2 Y": "m1m3_z_2",  # note these two have been
    "2 Z": "m1m3_y_2",  # switched pending SUMMIT-7911
    "3 X": "m1m3_x_3",
    "3 Y": "m1m3_y_3",
    "3 Z": "m1m3_z_3",
}
key_m2_dict = {
    "1 X": "m2_x_1",
    "1 Y": "m2_y_1",
    "1 Z": "m2_z_1",
    "2 X": "m2_x_2",
    "2 Y": "m2_z_2",
    "2 Z": "m2_y_2",
    "3 X": "m2_x_3",
    "3 Y": "m2_z_3",
    "3 Z": "m2_y_3",
    "4 X": "m2_x_4",
    "4 Y": "m2_y_4",
    "4 Z": "m2_z_4",
    "5 X": "m2_x_5",
    "5 Y": "m2_z_5",
    "5 Z": "m2_y_5",
    "6 X": "m2_x_6",
    "6 Y": "m2_z_6",
    "6 Z": "m2_y_6",
}


def vms_data_to_pandas(filename, vms_type, begin_time=None, end_time=None):
    """
    Converts VMS data in the given HDF5 file to a Pandas DataFrame.

    Args:
    filename: Path to the HDF5 file containing the VMS data.
    vms_type: The type of VMS data in the file. Must be "m1m3", "m2", or
      "rotator".
    begin_time: The start time of the data to include in the DataFrame. If None,
      all data will be included.
    end_time: The end time of the data to include in the DataFrame. If None, all
      data will be included.

    Returns:
    A Pandas DataFrame containing the VMS data.
    """
    if vms_type == "m1m3":
        key_dict = key_m1m3_dict
    elif vms_type == "m2":
        key_dict = key_m2_dict
    elif vms_type == "rotator":
        raise NotImplementedError
    else:
        raise ValueError("vms_type must be m1m3,m2, or rotator")

    f = h5py.File(filename, "r")
    times = f["timestamp"][::1]
    dkeys = "XYZ"

    data_dict = {}
    if (begin_time is not None) & (end_time is not None):
        sel = (times > begin_time) & (times < end_time)
    else:
        sel = np.ones(times.size).astype(bool)
    data_dict["times"] = times[sel]
    for key in key_dict.keys():
        # multiply values stored in hdf5 files by 2 in order to convert the acceleration values to mg
        data_dict[key_dict[key]] = f[key][::1][sel] * 2.0
    data_frame = pd.DataFrame(data_dict)
    for j in np.arange(int(len(key_dict) / 3)) + 1:
        data_frame[f"total_{j}"] = np.linalg.norm(
            data_frame[[f"{vms_type}_{i}_{j}" for i in ["x", "y", "z"]]].values, axis=1
        )

    return data_frame


def get_freq_psd(vals, timestep):
    """
    Calculates the frequency power spectrum of a signal.

    Args:
        vals (np.array): The signal values.
        timestep (float): The time step between samples.

    Returns:
        tuple: The frequencies and power spectral density.
    """

    # Remove the mean from the signal.

    meanval = np.mean(vals)
    signal = vals - meanval

    # Calculate the length of the signal.

    N = len(signal)

    # Calculate the power spectral density.

    psd = np.abs(np.fft.rfft(np.array(signal) * 1)) ** 2

    # Calculate the frequencies.

    frequencies = np.fft.rfftfreq(N, timestep)
    return (frequencies, psd)


def get_psd_and_dsd_for_vms(vals, timestep, min_freq=1, g=False):
    """
    Calculate the PSD and DSD from VMS data, and total displacement from DSD.

    Parameters:
    - vals (array-like): VMs in m/s^2 or milli-g if 'g' is True.
    - timestep (float): Time step between measurements in seconds.
    - min_freq (float, optional): Minimum frequency for calculations.
                                  Default is 1 Hz.
    - g (bool, optional): True if 'vals' are in milli-g units.
                          Default is False.

    Returns:
    - psds_df (DataFrame): DataFrame with frequencies ('freq'), acceleration
                           PSD ('accel_psd'), displacement PSD ('disp_psd'),
                           and cumulative displacement PSD ('int_disp').
    - total_displacement (float): Total displacement from vibration data.

    Note:
    PSD adjusted to m/s^2/Hz. 'vals' converted to m/s^2 if 'g' is True.
    """
    if g:
        vals = 1e-3 * 9.8 * vals

    # freq, accel_psd = signal.periodogram(vals, 1/timestep)
    # print("using scipy")
    freq, accel_psd = get_freq_psd(vals, timestep)

    sel = freq > min_freq
    freq = freq[sel]
    accel_psd = accel_psd[sel]

    accel_psd = accel_psd * np.mean(np.diff(freq))

    disp_psd_sq = accel_psd / ((2 * np.pi * freq) ** 4)

    int_displace_psd = np.sqrt(np.cumsum(disp_psd_sq))
    total_displacement = np.sqrt(np.sum(disp_psd_sq))

    psds_df = pd.DataFrame(
        {
            "freq": freq,
            "accel_psd": accel_psd,
            "disp_psd": np.sqrt(disp_psd_sq),
            "int_disp": int_displace_psd,
        }
    )

    return psds_df, total_displacement

In [ ]:
# Create a directory to save plots

plot_dir = "./plots"
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)

In [ ]:
# Directory containing VMS data
vms_dir = "/scratch/users/b/boutigny/vmsdata/12"
# Acquisition date of VMS data
vms_date = "2023-12-21"
# hdf5 file containing the VMS data
vms_m1m3_filename = os.path.join(vms_dir, "M1M3-" + vms_date + "T00:00.hdf")
print(f"We will read VMS data from file: {vms_m1m3_filename} ")

In [ ]:
# Read data from hdf file into pandas
vms_m1m3_data = vms_data_to_pandas(vms_m1m3_filename, vms_type="m1m3")

# Remove entries with null timestamps
vms_m1m3_data = vms_m1m3_data[vms_m1m3_data["times"] > 0]

In [ ]:
vms_m1m3_data

In [ ]:
vms_m1m3_data["times"] = Time(vms_m1m3_data["times"], format="unix").datetime

In [ ]:
%matplotlib inline
# First look at the raw VMS data - We plot only 1 sensor and 1 axis because the full dataset is very large and requires too much memory
# to be displayed
fig, ax = plt.subplots(1, 1, dpi=125, figsize=(8, 4))
key = "m1m3_z_3"
ax.plot(vms_m1m3_data["times"], vms_m1m3_data[key])
ax.set(ylabel="Acceleration (mg)", xlabel="Time", title=f"{vms_date} - {key}")
ax.xaxis.set_major_formatter(DateFormatter("%H-%M-%S"))
plt.setp(ax.get_xticklabels(), rotation=45)
fig.tight_layout()

In [ ]:
# Retrieve TMA events corresponding to the VMS acquisition date
dayObs = int(vms_date.replace("-", "", 2))
eventMaker = TMAEventMaker()
events = eventMaker.getEvents(dayObs)

# Get lists of slew events
slews = [e for e in events if e.type == TMAState.SLEWING]
print(f"Found {len(slews)} slews")

In [ ]:
# Filter the list of slews in order to keep the ones that are fully contained within the day corresponding to the hdf5 file.
date_min = Time(f"{vms_date} 00:00:00.00").unix
date_end = Time(f"{vms_date} 23:59:59.00").unix
sel_slews = [
    slews[i]
    for i in range(len(slews))
    if (slews[i].begin.unix > date_min and slews[i].end.unix < date_end)
]
print(f"Selected {len(sel_slews)} slews out of {len(slews)} for {dayObs}")

In [ ]:
pos_columns = ["xPosition", "yPosition", "zPosition"]
rot_columns = ["xRotation", "yRotation", "zRotation"]
all_columns = pos_columns + rot_columns

In [ ]:
client = EfdClient("usdf_efd")

In [ ]:
# Print slews amplitudes and speed in azimuth and declination
# The selected slew numbers will be stored in variables slew_azi and slew_ele
# by default they contain the slews with the largest amplitudes
max_delta_azi = -99
max_delta_ele = -99
for i_slew, slew in enumerate(sel_slews):
    df_ele = getEfdData(client, "lsst.sal.MTMount.elevation", event=slew)
    df_azi = getEfdData(client, "lsst.sal.MTMount.azimuth", event=slew)
    if len(df_ele) > 0:
        slew_delta_ele = df_ele["demandPosition"].max() - df_ele["demandPosition"].min()
        if slew_delta_ele > 30:
            print(
                f"Slew number: {i_slew} - Delta ele: {slew_delta_ele:.1f} degrees - Speed: {abs(df_ele['actualVelocity']).max()}"
            )
        if slew_delta_ele > max_delta_ele:
            max_delta_ele = slew_delta_ele
            slew_ele = i_slew
    if len(df_azi) > 0:
        slew_delta_azi = df_azi["demandPosition"].max() - df_azi["demandPosition"].min()
        if slew_delta_azi > 30:
            print(
                f"Slew number: {i_slew} - Delta azi: {slew_delta_azi:.1f} degrees - Speed: {abs(df_azi['actualVelocity']).max()})"
            )
        if slew_delta_azi > max_delta_azi:
            max_delta_azi = slew_delta_azi
            slew_azi = i_slew
print(
    f"Maximum amplitude slew in azimuth: {slew_azi} / {max_delta_azi:.1f} degrees - in elevation: {slew_ele} / {max_delta_ele:.1f} degrees"
)

In [ ]:
# This is the slew that we are going to analyze
# slew_select = slew_azi
slew_select = 79
# tma_axis = "azimuth"
tma_axis = "elevation"

# Add a small delta_t before and after the selected slew
delta_t = TimeDelta(10, format="sec")
start_slew = sel_slews[slew_select].begin - delta_t
end_slew = sel_slews[slew_select].end + delta_t
print(f"Selected slew - start: {start_slew.datetime64} - end: {end_slew.datetime64}")

In [ ]:
sel = (vms_m1m3_data["times"] > start_slew.datetime64) & (
    vms_m1m3_data["times"] < end_slew.datetime64
)

In [ ]:
df_azi = getEfdData(client, "lsst.sal.MTMount.azimuth", begin=start_slew, end=end_slew)
df_ele = getEfdData(
    client, "lsst.sal.MTMount.elevation", begin=start_slew, end=end_slew
)
print(len(df_azi), len(df_ele))

In [ ]:
# Plot a few quantities to check that we have selected a good quality slew

fig, ax = plt.subplots(1, 4, dpi=125, figsize=(15, 4))
ax[0].plot(vms_m1m3_data["times"][sel], vms_m1m3_data["m1m3_x_3"][sel])
ax[0].set(ylabel="accel (mg)", xlabel="Time", title=f"{vms_date} - Accel z_2")
ax[1].plot(df_azi.index, df_azi["actualPosition"], label="Azimuth")
ax[1].plot(df_ele.index, df_ele["actualPosition"], label="Declination")
ax[1].legend()
ax[1].set(ylabel="deg", xlabel="Time", title=f"{vms_date} - TMA Orientation")
ax[2].plot(df_azi.index, df_azi["actualTorque"])
ax[2].plot(df_azi.index, df_ele["actualTorque"])
ax[2].set(ylabel="N.m", xlabel="Time", title=f"{vms_date} - TMA Torque")
ax[2].plot(df_azi.index, df_azi["actualPosition"], label="Azimuth")
ax[2].plot(df_ele.index, df_ele["actualPosition"], label="Declination")
ax[2].legend()
ax[3].plot(df_azi.index, df_azi["actualVelocity"], label="Azimuth")
ax[3].plot(df_ele.index, df_ele["actualVelocity"], label="Declination")
ax[3].set(ylabel="deg/s", xlabel="Time", title=f"{vms_date} - TMA Speed")
ax[3].legend()
for i in range(4):
    ax[i].xaxis.set_major_formatter(DateFormatter("%H-%M-%S"))
    plt.setp(ax[i].get_xticklabels(), rotation=45)
fig.tight_layout()
fig.savefig(f"{plot_dir}/overview-{vms_date}-{slew_select}-.png")

In [ ]:
# PSD analysis

begin = sel_slews[slew_select].begin
end = sel_slews[slew_select].end

fig, ax = plt.subplots(3, 3, dpi=150, figsize=(8, 5))
for c in range(3):

    subdat_sel = (vms_m1m3_data["times"] > begin.datetime64) & (
        vms_m1m3_data["times"] < end.datetime64
    )
    subdat = vms_m1m3_data.loc[subdat_sel, :]
    for j, axis in enumerate("xyz"):
        key = f"m1m3_{axis}_{c+1}"
        psds_df, disp = get_psd_and_dsd_for_vms(
            subdat[key],
            np.mean(np.diff(Time(subdat["times"]).unix)),
            g=True,
            min_freq=1,
        )

        x = psds_df["freq"]
        y = psds_df["accel_psd"]
        ax[c][j].plot(x, y, zorder=9, lw=1)
        ax[c][j].set_xticks(np.arange(0, 110, 10))
        ax[c][j].set(
            ylabel="psd (m/$s^2$)/hz",
            xlabel="Frequency [Hz]",
            title=f"Sensor {c+1} - axis {axis}",
        )
        yticks = ticker.MaxNLocator(4)
        ax[c][j].yaxis.set_major_locator(yticks)
        # ax[c][j].set_ylim([0, 1e-5])
        ax[c][j].set_ylim([0, 1e-6])
        # ax[c][j].set_yscale("log")
# fig.suptitle(f"FCU speed: {percentv}%")
fig.tight_layout()
fig.savefig(f"{plot_dir}/psd-{vms_date}-{slew_select}-.png")

In [ ]:
# Spectrograms

begin = sel_slews[slew_select].begin
end = sel_slews[slew_select].end

df_azi = getEfdData(
    client,
    "lsst.sal.MTMount.azimuth",
    begin=begin,
    end=end,
)
df_ele = getEfdData(
    client,
    "lsst.sal.MTMount.elevation",
    begin=begin,
    end=end,
)

fig, ax = plt.subplots(3, 3, dpi=128, figsize=(12, 8))
ax2 = ax
for c in range(3):

    subdat_sel = (vms_m1m3_data["times"] > begin.datetime64) & (
        vms_m1m3_data["times"] < end.datetime64
    )
    subdat = vms_m1m3_data.loc[subdat_sel, :]
    fs = 1 / np.mean(np.diff(Time(subdat["times"]).unix))
    for j, axis in enumerate("xyz"):
        key = f"m1m3_{axis}_{c+1}"

        f, t, Zxx = stft(subdat[key], fs, "hamming", 128)

        ax[c][j].pcolormesh(t, f, np.abs(Zxx), vmin=0, vmax=1e-3, shading="gouraud")
        ax[c][j].set(
            xlabel="Time [s]",
            ylabel="Frequency [Hz]",
            title=f"Sensor {c+1} - axis {axis}",
        )
        ax2[c][j] = ax[c][j].twinx()
        ax2[c][j].plot(
            (df_azi.index - df_azi.index[0]).total_seconds(),
            df_azi["actualTorque"] / 100000,
            c="white",
            lw=1,
            ls="-",
            alpha=0.4,
        )
        ax2[c][j].plot(
            (df_ele.index - df_ele.index[0]).total_seconds(),
            df_ele["actualTorque"] / 100000,
            c="yellow",
            lw=1,
            ls="-",
            alpha=0.6,
        )
        ax2[c][j].set_yticks([])
fig.tight_layout()
fig.savefig(f"{plot_dir}/spectrogram-{vms_date}-{slew_select}-.png")

In [ ]:
# Explore the slew events in order to find a sufficiently long period of time during which the TMA is still
for i in range(len(sel_slews) - 1):
    t1 = sel_slews[i].end.unix
    t2 = sel_slews[i + 1].begin.unix
    delta_t = t2 - t1
    print(
        f"Between slew {i} and slew {i+1} the TMA is still during {delta_t:.0f} seconds"
    )

In [ ]:
delta_t = TimeDelta(20, format="sec")
begin_quiescent = sel_slews[10].end + delta_t
end_quiescent = sel_slews[11].begin - delta_t
print(begin_quiescent.datetime64, end_quiescent.datetime64)

sel_quiescent = (vms_m1m3_data["times"] > begin_quiescent.datetime64) & (
    vms_m1m3_data["times"] < end_quiescent.datetime64
)

df_azi = getEfdData(
    client, "lsst.sal.MTMount.azimuth", begin=begin_quiescent, end=end_quiescent
)
df_ele = getEfdData(
    client, "lsst.sal.MTMount.elevation", begin=begin_quiescent, end=end_quiescent
)
print(len(df_azi), len(df_ele))

In [ ]:
fig, ax = plt.subplots(1, 4, dpi=125, figsize=(15, 4))
ax[0].plot(
    vms_m1m3_data["times"][sel_quiescent], vms_m1m3_data["m1m3_x_3"][sel_quiescent]
)
ax[0].set(ylabel="accel (mg)", xlabel="Time", title=f"{vms_date} - Accel x_3")
ax[1].plot(df_azi.index, df_azi["actualPosition"], label="Azimuth")
ax[1].plot(df_ele.index, df_ele["actualPosition"], label="Declination")
ax[1].legend()
ax[1].set(ylabel="deg", xlabel="Time", title=f"{vms_date} - TMA Orientation")
ax[2].plot(df_azi.index, df_azi["actualTorque"])
ax[2].plot(df_ele.index, df_ele["actualTorque"])
ax[2].set(ylabel="N.m", xlabel="Time", title=f"{vms_date} - TMA Torque")
ax[2].plot(df_azi.index, df_azi["actualPosition"], label="Azimuth")
ax[2].plot(df_ele.index, df_ele["actualPosition"], label="Declination")
ax[2].legend()
ax[3].plot(df_azi.index, df_azi["actualVelocity"], label="Azimuth")
ax[3].plot(df_ele.index, df_ele["actualVelocity"], label="Declination")
ax[3].set(ylabel="deg/s", xlabel="Time", title=f"{vms_date} - TMA Speed")
ax[3].legend()
for i in range(4):
    ax[i].xaxis.set_major_formatter(DateFormatter("%H-%M-%S"))
    plt.setp(ax[i].get_xticklabels(), rotation=45)
fig.tight_layout()
fig.savefig(f"{plot_dir}/overview-still-{vms_date}-{slew_azi}-.png")

In [ ]:
begin = begin_quiescent
end = end_quiescent

fig, ax = plt.subplots(3, 3, dpi=150, figsize=(8, 5))
for c in range(3):

    subdat_sel = (vms_m1m3_data["times"] > begin.datetime64) & (
        vms_m1m3_data["times"] < end.datetime64
    )
    subdat = vms_m1m3_data.loc[subdat_sel, :]
    for j, axis in enumerate("xyz"):
        key = f"m1m3_{axis}_{c+1}"
        psds_df, disp = get_psd_and_dsd_for_vms(
            subdat[key],
            np.mean(np.diff(Time(subdat["times"]).unix)),
            g=True,
            min_freq=1,
        )

        x = psds_df["freq"]
        y = psds_df["accel_psd"]
        ax[c][j].plot(x, y, zorder=9, lw=1)
        ax[c][j].set_xticks(np.arange(0, 110, 10))
        ax[c][j].set(
            ylabel="psd (m/$s^2$)/hz",
            xlabel="Frequency [Hz]",
            title=f"Sensor {c+1} - axis {axis}",
        )
        yticks = ticker.MaxNLocator(4)
        ax[c][j].yaxis.set_major_locator(yticks)
        ax[c][j].set_ylim([0, 1e-5])
        # ax[c][j].set_yscale("log")
# fig.suptitle(f"FCU speed: {percentv}%")
fig.tight_layout()

In [ ]:
begin = begin_quiescent
end = end_quiescent

fig, ax = plt.subplots(3, 3, dpi=150, figsize=(8, 5))
for c in range(3):

    subdat_sel = (vms_m1m3_data["times"] > begin.datetime64) & (
        vms_m1m3_data["times"] < end.datetime64
    )
    subdat = vms_m1m3_data.loc[subdat_sel, :]
    fs = 1 / np.mean(np.diff(Time(subdat["times"]).unix))
    for j, axis in enumerate("xyz"):
        key = f"m1m3_{axis}_{c+1}"

        f, t, Zxx = stft(subdat[key], fs, "hamming", 128)

        ax[c][j].pcolormesh(t, f, np.abs(Zxx), vmin=0, vmax=1e-4, shading="gouraud")
        ax[c][j].set(
            xlabel="Time [s]",
            ylabel="Frequency [Hz]",
            title=f"Sensor {c+1} - axis {axis}",
        )
# fig.suptitle(f"FCU speed: {percentv}%")
fig.tight_layout()
fig.savefig(f"{plot_dir}/spectrogram-still-{vms_date}-{slew_azi}-.png")